In [3]:
from qonnx.util.basic import get_by_name
from qonnx.transformation.create_generic_partitions import PartitionFromDict
from qonnx.core.modelwrapper import ModelWrapper
import os

model = ModelWrapper("/home/pgeel/FINNv0.8.1_repo/build_KV260/finn/notebooks/MSC_Git_files/mobilenet_v1/models/mobilenet_streamline.onnx")
splits = []
for n in model.graph.node:
    if n.op_type == 'MultiThreshold':
        splits.append(n.output[0])

def custom_step_partition(stop_node):
    output_dir = os.path.join("CPU_testing",stop_node)
    print(output_dir)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    upstream_0 = model.find_upstream(stop_node, lambda x: x.name == "Conv_0")
    wanted_nodes = []
    unwanted_nodes = []
    for ind, node in enumerate(model.graph.node):
        found_0 = get_by_name(upstream_0, node.name, "name") is not None
        if found_0:
            wanted_nodes.append(ind)
        else:
            unwanted_nodes.append(ind)
    parent=model.transform(PartitionFromDict(
        partitioning={ 0 : wanted_nodes, 1 : unwanted_nodes }, 
        partition_dir= output_dir
    ))
#     return ModelWrapper(output_dir+"/partition_1.onnx")

for s in splits:
    custom_step_partition(s)



CPU_testing/MultiThreshold_0_out0
CPU_testing/MultiThreshold_1_out0
CPU_testing/MultiThreshold_2_out0
CPU_testing/MultiThreshold_3_out0
CPU_testing/MultiThreshold_4_out0
CPU_testing/MultiThreshold_5_out0
CPU_testing/MultiThreshold_6_out0
CPU_testing/MultiThreshold_7_out0
CPU_testing/MultiThreshold_8_out0
CPU_testing/MultiThreshold_9_out0
CPU_testing/MultiThreshold_10_out0
CPU_testing/MultiThreshold_11_out0
CPU_testing/MultiThreshold_12_out0
CPU_testing/MultiThreshold_13_out0
CPU_testing/MultiThreshold_14_out0
CPU_testing/MultiThreshold_15_out0
CPU_testing/MultiThreshold_16_out0
CPU_testing/MultiThreshold_17_out0
CPU_testing/MultiThreshold_18_out0
CPU_testing/MultiThreshold_19_out0
CPU_testing/MultiThreshold_20_out0
CPU_testing/MultiThreshold_21_out0
CPU_testing/MultiThreshold_22_out0
CPU_testing/MultiThreshold_23_out0
CPU_testing/MultiThreshold_24_out0
CPU_testing/MultiThreshold_25_out0
CPU_testing/MultiThreshold_26_out0


In [48]:
from qonnx.core.modelwrapper import ModelWrapper
from onnxruntime_extensions import get_library_path, PyOp, onnx_op, PyOrtFunction
import os
import onnxruntime as ort
import onnx
import numpy as np
import time
from onnx import helper
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat
import pandas as pd

# CPU_TESTING
rt = {}
so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())

def set_multithreshold_default(model):#,save_model):
    '''
    Pass a modelproto model and the save file
    '''
    model = model.transform(DoubleToSingleFloat())
    new_attr = [helper.make_attribute("out_scale", 1.0),
                helper.make_attribute("out_bias", 0.0),
                helper.make_attribute("data_layout","NCHW")]

    for n in model.graph.node:
        if n.op_type == "MultiThreshold":
            out_scale,bias,datalayout = False,False,False
            for na in n.attribute:
                if na.name == "out_scale": out_scale = True
                if na.name == "out_bias": bias = True
                if na.name == "data_layout": datlayout = True
            if not out_scale: n.attribute.append(new_attr[0])
            if not bias: n.attribute.append(new_attr[1])
            if not datalayout: n.attribute.append(new_attr[2])

            n.domain = "ai.onnx.contrib"
#     model.save(save_model)
    return model

def size_model(model):
    cnt = 0
    for n in model.graph.node:
        cnt +=1
    return cnt
for j in os.listdir("CPU_testing"):
    if os.path.isdir(os.path.join("CPU_testing",os.path.relpath(j))):
        file = os.path.join("CPU_testing",os.path.relpath(j),"partition_1.onnx")
    #     Place onnx runtime code here, need to generate dummy data to pass to the onnx graph.
        model = ModelWrapper(file)
        model = set_multithreshold_default(model)

        sess = ort.InferenceSession(model.model.SerializeToString(),so)
        x = np.random.randint(0,255,sess.get_inputs()[0].shape).astype(np.float32)
        start = time.time()
        res = sess.run([],{sess.get_inputs()[0].name:x})
        end = time.time()
        rt[size_model(model)] = (end-start)*1e3
rt

{45: 209.05160903930664,
 31: 138.76914978027344,
 23: 100.30174255371094,
 33: 154.87194061279297,
 53: 290.3599739074707,
 15: 50.423383712768555,
 19: 72.51334190368652,
 5: 1.667022705078125,
 57: 352.4923324584961,
 13: 34.73925590515137,
 51: 284.4200134277344,
 39: 185.4090690612793,
 21: 83.70494842529297,
 37: 156.16583824157715,
 41: 190.5691623687744,
 7: 18.42331886291504,
 47: 236.69910430908203,
 17: 58.408498764038086,
 11: 32.36246109008789,
 43: 206.96282386779785,
 9: 24.120330810546875,
 27: 121.88911437988281,
 35: 153.36918830871582,
 49: 260.1919174194336,
 25: 101.85074806213379,
 29: 124.22013282775879,
 55: 341.86482429504395}

In [49]:
import json
with open("CPU_testing/runtimes.json",'w') as f:
    json.dump(rt,f)